<a href="https://colab.research.google.com/github/ChenghenChen/Slum/blob/main/Sat_image_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import cv2
import numpy as np
from segmentation_models import Unet  # 假設使用 Unet 模型，您也可以根據需要選擇其他模型
from segmentation_models import get_preprocessing

# 定義函式來使用 Segment Anything 進行圖像分割
def segment_image(image):
    # 加載預訓練的模型
    model = Unet('resnet34', classes=3, activation='softmax')
    model.load_weights('path_to_segmentation_model_weights.h5')  # 指定模型權重的路徑

    # 預處理圖像
    preprocessing_fn = get_preprocessing('resnet34')
    image = preprocessing_fn(image)

    # 進行圖像分割
    segmented_image = model.predict(image[np.newaxis, ...])

    return segmented_image

# 定義函式來檢測房屋輪廓並計算面積
def detect_and_measure_houses(image):
    # 將圖片轉換為灰度圖
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 二值化處理
    ret, thresh = cv2.threshold(gray, 127, 255, 0)

    # 查找輪廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 創建一個列表來存儲房屋的面積
    house_areas = []

    # 遍歷每個輪廓
    for contour in contours:
        # 計算輪廓的面積
        area = cv2.contourArea(contour)
        # 如果面積太小，可能不是房屋，忽略
        if area < 100:
            continue
        # 將房屋的面積添加到列表中
        house_areas.append(area)

    return house_areas

# 讀取衛星照片
image_path = '/content/satellite_image/1212.jpg'
image = cv2.imread(image_path)

# 使用 Segment Anything 進行圖像分割
segmented_image = segment_image(image)

# 檢測房屋輪廓並計算面積
house_areas = detect_and_measure_houses(image)

# 分類房屋
small_houses = sum(1 for area in house_areas if area < 100)  # 假設小房屋的面積小於100平方米
medium_houses = sum(1 for area in house_areas if 100 <= area < 200)  # 假設中房屋的面積介於100至200平方米之間
large_houses = sum(1 for area in house_areas if area >= 200)  # 假設大房屋的面積大於等於200平方米

# 顯示結果
print("小房屋數量:", small_houses)
print("中房屋數量:", medium_houses)
print("大房屋數量:", large_houses)


ModuleNotFoundError: No module named 'segmentation_models'